In [1]:
import os
from pathlib import Path

config_path = Path.home() / ".fiftyone" / "config.global_mongodb.json"
os.environ["FIFTYONE_CONFIG_PATH"] = str(config_path)

In [2]:
import evaluate
import logging
from seametrics.payload import PayloadProcessor

logging.basicConfig(level=logging.INFO)

processor = PayloadProcessor(
     dataset_name="SAILING_DATASET_QA",
     gt_field="ground_truth_det",
     models=["yolov5n6_RGB_D2304-v1_9C"],
     tracking_mode=False,
     sequence_list=["Trip_14_Seq_1", "Trip_14_Seq_2"],
     data_type="rgb",
)

INFO:seametrics.payload:Processing dataset: SAILING_DATASET_QA
INFO:seametrics.payload:GT field: ground_truth_det
INFO:seametrics.payload:Models: ['yolov5n6_RGB_D2304-v1_9C']
INFO:seametrics.payload:Tracking mode: False
INFO:seametrics.payload:Sequence list: ['Trip_14_Seq_1', 'Trip_14_Seq_2']
INFO:seametrics.payload:Data type: rgb
INFO:seametrics.payload:Excluded classes: ['ALGAE', 'BRIDGE', 'HARBOUR', 'WATERTRACK', 'SHORELINE', 'SUN_REFLECTION', 'UNSUPERVISED', 'WATER', 'TRASH', 'OBJECT_REFLECTION', 'HORIZON']
INFO:seametrics.payload:Using slice: ['rgb']
Processing sequences: 100%|██████████| 2/2 [00:01<00:00,  1.83it/s]
INFO:seametrics.payload:Initialized PayloadProcessor for dataset: SAILING_DATASET_QA


In [3]:
for name, sequence in processor.payload.sequences.items():
    print(f"Sequence: {name} stats")
    print(f"Resolution: {sequence.resolution}")
    for field_name in sequence.field_names:
        print(f"{field_name} has {len(sequence[field_name])} elements")
    print()

Sequence: Trip_14_Seq_1 stats
Resolution: Resolution(height=720, width=1280)
ground_truth_det has 22 elements
yolov5n6_RGB_D2304-v1_9C has 22 elements

Sequence: Trip_14_Seq_2 stats
Resolution: Resolution(height=720, width=1280)
ground_truth_det has 51 elements
yolov5n6_RGB_D2304-v1_9C has 51 elements



In [14]:
from seametrics.detection.utils import payload_to_det_metric

module = evaluate.load("/Users/kevinserrano/HF/det-metrics/det-metrics.py",
                       iou_threshold=[0.5])
predictions, ground_truth = payload_to_det_metric(processor.payload)
# convert all numpy arrays to lists
module.add(prediction=predictions, reference=ground_truth)

In [17]:
module.add_from_payload(processor.payload)
module.compute()

                                             METRIC     tp,     fp,     fn,    dup,    pr,    re,    f1,   supp,    fpi,  nImgs
@[ IoU=0.50      | area=      all | maxDets=100 ] =    122,    220,     86,      0,  0.36,  0.59,  0.44,    208,     64,    146


{'all': {'range': [0, 10000000000.0],
  'iouThr': '0.50',
  'maxDets': 100,
  'tp': 122,
  'fp': 220,
  'fn': 86,
  'duplicates': 0,
  'precision': 0.3567251461988304,
  'recall': 0.5865384615384616,
  'f1': 0.4436363636363636,
  'support': 208,
  'fpi': 64,
  'nImgs': 146}}